## Lighthouse Labs
### W02D1 SQL

Instructor: Socorro Dominguez  
March 08, 2021

## Agenda
1. Context and Landscape (40 mins)
2. Break (10 mins)
3. Writing SELECT statements (60 mins)
    * Filtering, ordering, limiting, etc.
    * Joining tables
    * Grouping records
    * Aggregate functions

## Why databases?

* Organized way to store data and relate data to each other.
* Enforces strict structure and relationships (forces you to keep data clean)
    - Makes it easier to train Machine Learning algorithms
* Enables data governance (e.g., availability, usability, integrity and security of data)
* Efficient retrieval of structured data
* Can store lots of data

_"A company cannot be managed with Excel spreadsheets"_

## Why SQL?

* A lot of data is still stored in SQL databases
* Structured Query Language (SQL) still the most proficient tool to investigate, filter, slice and dice your data

## NoSQL databases
* There exists NoSQL databases (e.g., MongoDB)
* Want to learn more? https://www.mongodb.com/nosql-explained

### Why are we learning SQL?

- Public dataset format has less common use-case.
- Internal database querying
    - May contain information you don’t want
    - May want to train your model on combinations/aggregations of fields from various tables
- Data exploration through simple operations on different groups/subsets
- Better understand transformations on data
- Mentioned in almost every data science job posting
    - Data science jobs ask for a ton of skills, good to have exposure to all for interview purposes, even if they won’t all realistically be necessary

### Challenges when writing SQL

- Declarative (SQL) vs. imperative (Python). Basically, no control flow
- Long, nested queries with many variable names
    - Vs. imperative programming where most readable programs break logic up into multiple steps
- Many things happening concurrently in a single statement, order not explicit
    - Vs. imperative programming where code executes line-by-line
- Debugging is more difficult due to the above
    - Can help to break a complex query down into steps and test those out first. Incremental approach to writing the query
    
    
To review the fundamentals: https://www.w3schools.com/sql/

### Database schemas

- How do the different tables relate to each other?
- Arguably most difficult part of relational databases is designing the schema
    - Less of a concern for data scientists — not our job!
- For our purposes, useful to understand table structure of a database to know how to write our queries (e.g. what tables to join)
- Common design principles
    - Star schema
    - Snowflake shema


**Resources:**
* https://www.guru99.com/star-snowflake-data-warehousing.html
* https://en.wikipedia.org/wiki/Snowflake_schema
* https://en.wikipedia.org/wiki/Star_schema

### Star Schema

<img src="star-schema.png" width=300>

* Every dimension represented by only one dimension table
* Dimension table contains set of attributes
* Dimension tables are joined to the fact table using a foreign key
* Dimension tables **are not** joined to each other
* Fact table contains key and measure

### Snowflake Schema

<img src="snowflake-schema.png" width=700>

* Extension of the star schema by adding extra dimensions
* Dimension tables are "normalized", splitting data into additional tables
* What does it mean to [normalize data](https://docs.microsoft.com/en-us/office/troubleshoot/access/database-normalization-description#:~:text=Normalization%20is%20the%20process%20of,eliminating%20redundancy%20and%20inconsistent%20dependency.)?

## Schema comparison

**Star schema:**
* Easy to understand
* Dimension tables are not normalized - country ID does not have country lookup table
* Widely supported by BI tools

**Snowflake schema:**
* Easy to implement and "grow"
* Lower query performance with multiple tables
* More difficult to maintain with more lookup tables


## RDBMS Landscape


<img src="rdbms.png" width=400>

* Relational Database Management System
* Software system that enables users to define, create, maintain and control access to the database
* Many, many, many vendors

### RDBMS Vendors

* **Closed source** (i.e., you have to pay)
    * Vendors: Oracle, SQL Server (Microsoft), IBM DB2, Microsoft Access - local small databases
    * Could come with integrations and services that make things easier  
    
<br>

* **Open source** (i.e., free)
    * MySQL, PostgreSQL, SQLite, MariaDB
    * Good developer community makes these great options
    * [This website](https://www.digitalocean.com/community/tutorials/sqlite-vs-mysql-vs-postgresql-a-comparison-of-relational-database-management-systems#:~:text=SQLite%20is%20a%20self%2Dcontained,even%20in%20low%2Dmemory%20environments.) offers a good comparison of open source systems options.

## Why SQLite?

* Not directly comparable to client/server SQL database engines such as MySQL, Oracle, PostgreSQL, or SQL Server
* Used as on-disk file format for desktop applications
* Great to learn on to get a hang of SQL

For more information: https://www.sqlite.org/whentouse.html

## Why PostGres? 

* Created by scientists from the University of California at Berkeley
* Open source nature makes it easy to upgrade or extend
* High compliance to the SQL standard
* Offers its users a huge (and growing) number of functions allowing programmers to create new applications, better protect data integrity, and developers build resilient and secure environments
* Can easily run it on Windows, Mac OS X, and almost all Linux and Unix distributions
* MySQL would be a good choice too
* We use will use `psql`


## Make sure everyone is set up before break...

1. Install PostGreSQL: https://www.datacamp.com/community/tutorials/installing-postgresql-windows-macosx
* From your terminal: `pip install psycopg2`
* Open a jupyter notebook and run `import psycopg2`to make sure it works 

To get a database going: 
1. Open pgsql shell
* Log in (should be all "enter" if you left it at default)
* Set a password (REMEMBER THIS!)
* `\l` to see what databases already exist
* `CREATE DATABASE drinks;`
* `\l` to see that it was created

To write to a database:

(You will need the `drinks.sql` file in the same directory as this notebook)

In [3]:
import psycopg2
import config as cfg

postgres_pwd = cfg.postgres['password'] # This is the password you entered when you set up PGSQL

con = psycopg2.connect(database='drinks', user='postgres', password=postgres_pwd,
                       host='127.0.0.1', port='5432')  # This should work if you left everything as default

cur = con.cursor()

#cur.execute(open('drinks.sql', 'r').read())  # Uncomment this the first time you run the db

# Break

- Take 10 minutes, come back at ...


## Demo
* Show you how to query data from a database
* Statements that start with `SELECT`

## Intro to the sample database we're using - Drinks

Database schema:

* has_on_menu(<em><ins>bar</ins></em>: string, <em><ins>drink_id</ins></em>: string, <em><ins>price</ins></em>: real)


* orders(<em><ins>person</ins></em>: string, <em><ins>date</ins></em>: string, bar, <em><ins>drink_id</ins></em>: string, quantity: integer)


* drink_info(<em><ins>drink_id</ins></em>: string, <em><ins>type</ins></em>: string)

The `drink_info` relation specifies for each `drink_id` what type of drink it is, e.g., beer, wine, etc.

## Setting up the connection to our database

In [4]:
# make necessary imports and set up environmental variables
import psycopg2
import pandas as pd

postgres_pwd = 'postgres'

In [5]:
# connect to the database
con = psycopg2.connect(database='drinks', user='postgres', password=postgres_pwd,
                       host='127.0.0.1', port='5432')

query = con.cursor()

## Executing our first query

Get the first 5 rows from orders

In [6]:
# execute a simple query
query.execute(
"""
SELECT *
FROM orders
LIMIT 5;
""")

response = query.fetchall()

for row in response:
    print(row)

('person 1', '2016-10-16', 'bar 9', 'drink 4', 1)
('person 1', '2016-10-16', 'bar 9', 'drink 44', 1)
('person 1', '2016-10-22', 'bar 19', 'drink 1', 4)
('person 1', '2016-10-22', 'bar 19', 'drink 9', 1)
('person 1', '2016-10-22', 'bar 19', 'drink 42', 2)


An easier way to see the data...

In [7]:
pd.DataFrame(response)

,0,1,2,3,4
0,person 1,2016-10-16,bar 9,drink 4,1
1,person 1,2016-10-16,bar 9,drink 44,1
2,person 1,2016-10-22,bar 19,drink 1,4
3,person 1,2016-10-22,bar 19,drink 9,1
4,person 1,2016-10-22,bar 19,drink 42,2


## Quick recap for SQL

The basic format is
<img src='select-statement.png' width=200>

What would the following tell us with our database?
```
SELECT *
FROM orders
WHERE bar='bar 9'
```

---
Reminder - our database:

<img src='our-db.png' width=400 align='left'>

## Other commands

Use a cheat sheet!
<img src='sql-cheat-sheet.png' width=700>

## More on joins

- `(INNER) JOIN`: Returns records that have matching values in both tables
- `LEFT (OUTER) JOIN`: Returns all records from the left table, and the matched records from the right table
- `RIGHT (OUTER) JOIN`: Returns all records from the right table, and the matched records from the left table
- `FULL (OUTER) JOIN`: Returns all records when there is a match in either left or right table

<img src='joins.png'>

## Let's work through some problems

### Question 1:
Get the bar name and average price of each bar

What do you think? What table do we need? What calculation do we need?

---
Reminder - our database:

<img src='our-db.png' width=400 align='left'>

In [8]:
# Carly
query.execute(
"""
SELECT bar, AVG(price) 
FROM has_on_menu 
GROUP BY has_on_menu.bar
""")

response = query.fetchall()
pd.DataFrame(response)

,0,1
0,bar 14,93.438857
1,bar 8,67.919044
2,bar 16,121.364365
3,bar 15,90.993793
4,bar 12,152.071032
5,bar 6,27.682568
6,bar 17,107.329703
7,bar 9,52.381706
8,bar 7,61.567487
9,bar 18,105.214779


In [9]:
query.execute(
"""
SELECT bar, AVG(price) AS avg_price
FROM has_on_menu
GROUP BY bar
""")

response = query.fetchall()

pd.DataFrame(response)

,0,1
0,bar 14,93.438857
1,bar 8,67.919044
2,bar 16,121.364365
3,bar 15,90.993793
4,bar 12,152.071032
5,bar 6,27.682568
6,bar 17,107.329703
7,bar 9,52.381706
8,bar 7,61.567487
9,bar 18,105.214779


In [10]:
print(response)

[('bar 14', 93.43885736465454), ('bar 8', 67.91904382705688), ('bar 16', 121.36436504125595), ('bar 15', 90.99379272460938), ('bar 12', 152.07103157043457), ('bar 6', 27.682567596435547), ('bar 17', 107.32970309257507), ('bar 9', 52.38170552253723), ('bar 7', 61.56748652458191), ('bar 18', 105.2147786617279), ('bar 11', 31.56255849202474), ('bar 19', 18.22443421681722), ('bar 3', 129.80951288768225), ('bar 5', 100.60385791460673), ('bar 20', 86.51315768559773), ('bar 10', 60.57263496943882), ('bar 1', 83.67851856776646), ('bar 13', 150.56043901443482), ('bar 4', 49.25097733736038), ('bar 2', 60.25870447158813)]


In [11]:
colnames = [desc[0] for desc in query.description]
print(colnames)

['bar', 'avg_price']


### Question 2:
Get the bars with the top 5 average prices.

What do you think? How can we adapt the code we did before?

---
Reminder - our database:

<img src='our-db.png' width=400 align='left'>

In [12]:
query.execute(
"""
SELECT bar, AVG(price) AS avg_price
FROM has_on_menu
GROUP BY bar
ORDER BY avg_price DESC
LIMIT 5
""")

response = query.fetchall()

pd.DataFrame(response)

,0,1
0,bar 12,152.071032
1,bar 13,150.560439
2,bar 3,129.809513
3,bar 16,121.364365
4,bar 17,107.329703


In [13]:
colnames = [desc[0] for desc in query.description]
print(colnames)

['bar', 'avg_price']


### Question 3:
Which bar sells the cheapest drink? Which drink and what's the price?

---
Reminder - our database:

<img src='our-db.png' width=400 align='left'>

In [14]:
query.execute(
"""
SELECT bar, drink_id, price
FROM has_on_menu
ORDER BY price ASC
LIMIT 1;
""")

response = query.fetchall()

pd.DataFrame(response)

,0,1,2
0,bar 18,drink 43,3.477886


### Question 4:
What is the number of beers sold by each bar?

(If you interpret it to be "sold" as in available for sale, then you need `has_on_menu`; if you interpret "sold" as in a sale was made, then you need `orders`)

Hint: you need two tables here

---
Reminder - our database:

<img src='our-db.png' width=400 align='left'>

In [15]:
query.execute(
"""
SELECT orders.bar, SUM(orders.quantity) as beers_sold from orders
JOIN drink_info ON drink_info.drink_id = orders.drink_id
WHERE drink_info.type = 'beer'
GROUP BY orders.bar
ORDER BY beers_sold DESC
"""
)

response = query.fetchall()

pd.DataFrame(response)

,0,1
0,bar 20,176
1,bar 5,111
2,bar 3,108
3,bar 11,80
4,bar 2,79
5,bar 10,66
6,bar 17,36


In [16]:
query.execute(
"""
SELECT o.bar, SUM(o.quantity) total_beer
    FROM orders o, drink_info d
    WHERE o.drink_id = d.drink_id 
        AND d.type = 'beer'
    GROUP BY bar
    ORDER BY total_beer;
""")

response = query.fetchall()

pd.DataFrame(response)

,0,1
0,bar 17,36
1,bar 10,66
2,bar 2,79
3,bar 11,80
4,bar 3,108
5,bar 5,111
6,bar 20,176


### Challenge question

For each person, find the bar they visit, and the type(s) and price(s) of the drink(s) they drink during those visits.

---
Reminder - our database:

<img src='our-db.png' width=400 align='left'>

In [17]:
query.execute(
"""
SELECT o.person, o.bar, d.type, h.price
    FROM orders o
    JOIN has_on_menu h ON (o.drink_id = h.drink_id AND o.bar = h.bar) 
    JOIN drink_info d ON o.drink_id = d.drink_id;
""")

response = query.fetchall()

pd.DataFrame(response)

,0,1,2,3
0,person 1,bar 9,cocktail,26.487211
1,person 1,bar 9,vodka,99.061460
2,person 1,bar 19,cocktail,11.775652
3,person 1,bar 19,rum,18.195812
4,person 1,bar 19,whisky,34.507183
...,...,...,...,...
1605,person 100,bar 2,cocktail,18.345350
1606,person 100,bar 2,beer,7.545805
1607,person 100,bar 2,rum,6.449759
1608,person 100,bar 15,wine,144.248120


### Making sure we close off the connection

In [18]:
query.close()

## Additional resoures

* What does it mean to [normalize data](https://docs.microsoft.com/en-us/office/troubleshoot/access/database-normalization-description#:~:text=Normalization%20is%20the%20process%20of,eliminating%20redundancy%20and%20inconsistent%20dependency.)? 